In [10]:
# from IPython import display
import grequests
import requests
from lxml import html
import pandas as pd
import re
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
email = config['login']['email']
password = config['login']['password']


pd.set_option('display.max_rows', 100)

df = pd.DataFrame()

url = 'https://profitus.lt/users/login'

resp = ''

login_data = {}

def get_data():
  urls = []
  with requests.session() as client:
    client.get(url)
    csrftoken = client.cookies['csrfToken']
    login_data = dict(email=email, password=password, _csrfToken=csrftoken, next='/')
    r = client.post(url, data=login_data, headers=dict(Referer=url))
    resp = client.get('https://www.profitus.lt/secondary-market')
    tree = html.fromstring(resp.text)
    next_url = 'https://profitus.lt' + tree.xpath('//a[@class="page-link-p fw-700"]')[-1].values()[0]

    number_of_pages = int(re.search("\d+", next_url).group())

    for i in range(1, number_of_pages+1):
      urls.append(f'https://www.profitus.lt/secondary-market?page={i}')
      
    reqs = [grequests.get(link, headers=dict(Referer=url), session=client) for link in urls]
    resp = grequests.map(reqs)
    return resp

def parse_data(resp):
  global df
  for r in resp:
    print(r.url)
    df = pd.concat([df, pd.read_html(r.text)[0]])

  print('scraping is over!')


from time import time

current_time = time()

resp = get_data()
parse_data(resp)

print("Time taken: ", time() - current_time)

df = df.drop(['Unnamed: 0', 'Unnamed: 9'], axis=1)

df['Projekto pavadinimas'] = df['Projekto pavadinimas'].map(lambda x: x.lstrip('Projekto pavadinimas:&nbsp '))
df['Reitingas'] = df['Reitingas'].map(lambda x: x.lstrip('Reitingas:&nbsp '))
df['Likęs terminas'] = df['Likęs terminas'].map(lambda x: x.lstrip('Likęs terminas:&nbsp ').rstrip('mėn.'))

# df['Likęs terminas'] = df['Likęs terminas'].map(lambda x: x.split('/')[0]).astype(int) + 1 # 1 month and 29 days is considered as 1 month, we're calculating worst example possible. only those will be worth to buy
df['Likęs terminas'] = df['Likęs terminas'].map(lambda x: x.split('/')[0]).astype(int) # 1 month and 29 days is considered as 1 month, we're calculating worst example possible. only those will be worth to buy


df['Likusi suma'] = df['Likusi suma'].map(lambda x: x.lstrip('Likusi suma:&nbsp'))

df['Likusi suma'] = df['Likusi suma'].str.replace('€', '')
df['Likusi suma'] = df['Likusi suma'].str.replace(',', '').astype(float)

df['Palūkanų norma'] = df['Palūkanų norma'].map(lambda x: x.lstrip('Palūkanų norma:&nbsp'))

df['Palūkanų norma'] = df['Palūkanų norma'].str.replace('%', '').astype(float)


df['Statusas'] = df['Statusas'].map(lambda x: x.lstrip('Statusas:&nbsp'))

df['Statusas'] = df['Statusas'].str.replace('Aktyvus', '1')
df['Statusas'] = df['Statusas'].str.replace('Vėluoja', '2').astype(int)



df['Likusi gautina sumai'] = df['Likusi gautina sumai'].map(lambda x: x.lstrip('Likusi gautina suma:&nbsp '))

df['Likusi gautina sumai'] = df['Likusi gautina sumai'].str.replace('€', '')
df['Likusi gautina sumai'] = df['Likusi gautina sumai'].str.replace(',', '').astype(float)

df['Pardavimo kaina'] = df['Pardavimo kaina'].map(lambda x: x.lstrip('Pardavimo kaina:&nbsp '))

df['Pardavimo kaina'] = df['Pardavimo kaina'].str.replace('€', '')
df['Pardavimo kaina'] = df['Pardavimo kaina'].str.replace(',', '').astype(float)

# df['Realios palūkanos'] = (df['Likusi gautina sumai'] - df['Pardavimo kaina']) * 100 / df['Pardavimo kaina'] / (df['Likęs terminas']/12)
import numpy as np
df[df['Likęs terminas'] < 1] = 1
df['Realios palūkanos'] = (np.power(df['Likusi gautina sumai'] / df['Pardavimo kaina'], 1/(df['Likęs terminas']/12)) - 1 ) * 100

# print(df[(df['Palūkanų norma'] - df['Realios palūkanos']) < 0]) # basically we're trying to find projects that have largest interest rate, than
# print(df[df['Likęs terminas'] < 1].sort_values('Likęs terminas', ascending=True))
from IPython.display import display
display(df[(df['Palūkanų norma'] - df['Realios palūkanos']) < 0])
df.sort_values('Realios palūkanos', ascending=False).head(100)

https://www.profitus.lt/secondary-market?page=1
https://www.profitus.lt/secondary-market?page=2
https://www.profitus.lt/secondary-market?page=3
https://www.profitus.lt/secondary-market?page=4
https://www.profitus.lt/secondary-market?page=5
https://www.profitus.lt/secondary-market?page=6
https://www.profitus.lt/secondary-market?page=7
https://www.profitus.lt/secondary-market?page=8
scraping is over!
Time taken:  2.9067397117614746


,Projekto pavadinimas,Reitingas,Likęs terminas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Realios palūkanos


,Projekto pavadinimas,Reitingas,Likęs terminas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Realios palūkanos


,Projekto pavadinimas,Reitingas,Likęs terminas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Realios palūkanos
2,#395 Saurenta plėtra,C+,9,530.00,8.5,1,574.80,545.00,7.356170
1,#357 Namas Antakalnyje,C+,6,216.49,8.0,2,228.59,221.00,6.986729
1,#312 Palėpės rekonstrukcija Kauno centre II,B-,3,100.00,7.5,1,103.70,102.00,6.835193
3,#347 Užupio butai,B,6,100.00,8.0,2,105.35,102.00,6.676495
3,#396 Lapino namai IV,B-,9,505.00,8.0,1,545.18,520.00,6.507967
0,#411 Antakalnio vakarai,D,10,500.00,12.0,1,559.67,535.00,5.558666
5,#421 Butas Pylimo g.,B+,7,100.00,7.0,1,105.22,102.04,5.401730
4,"#404 Kvartalas „Baltai"" II",B-,9,1915.90,9.5,1,2096.91,2020.00,5.108514
0,#348 Radastų namai II,B-,6,1000.00,8.0,1,1055.89,1030.00,5.090366
2,#428 Namai Bališkių pušyne,B-,10,1219.63,8.5,1,1322.73,1270.00,5.002824


C:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


,Projekto pavadinimas,Reitingas,Likęs terminas,Likusi suma,Palūkanų norma,Statusas,Likusi gautina sumai,Pardavimo kaina,Realios palūkanos
1,#270 Komfortiški soduose II,B-,0,498.68,8.0,2,498.68,511.0,-100.0


C:\Users\adoma\AppData\Local\Programs\Python\Python39\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
